# Использование Git

---

# Подготовка

Git различает изменения, сделанные разными пользователями, поэтому следует себя идентифицировать.

In [ ]:
git config --global user.name 'Ivan Dmitrievsky'
git config --global user.email 'ivan.dmitrievsky@gmail.com'

---

# Описание задачи

Имеется MAP файл, в котором каждая строчка описывает один SNP в датасете. Каждая строчка состоит из четырёх столбцов, разделённых символом табуляции:

* Chromosome id (1-22, X, Y or 0)
* SNP id (string)
* Genetic distance (morgans)
* Base-pair position (bp units)

## Пример

```
1  rs123456  0  1234555
1  rs234567  0  1237793
1  rs224534  0  1237697
...
2  rs233556  0  91536
...
```

Хочется узнать, сколько SNP относится к каждой из хромосом.

---

# Создание проекта

In [1]:
pwd

/Users/idmit/Documents/dev/sandbox/git_workshop


In [2]:
mkdir -p gitstarter/count
cd gitstarter/count

---

# Создание репозитория

Чтобы превратить рабочий каталог в git репозиторий используется следующая команда.

In [3]:
git init

Initialized empty Git repository in /Users/idmit/Documents/dev/sandbox/git_workshop/gitstarter/count/.git/


Команда для просмотра состояния репозитория:

In [4]:
git status

On branch master

Initial commit

nothing to commit (create/copy files and use "git add" to track)


In [4]:
ls -lah

total 0
drwxr-xr-x  3 idmit  staff   102B Feb 28 17:05 .
drwxr-xr-x  3 idmit  staff   102B Feb 28 17:05 ..
drwxr-xr-x  9 idmit  staff   306B Feb 28 17:05 .git


Git создал свой служебный каталог `.git`. Родительская директория перестанет быть репозиторием, если удалить `.git`. 

---

# Входные данные

In [5]:
curl -O 'https://s3.amazonaws.com/3kricegenome/reduced/NB-core_v4.map.gz'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6040k  100 6040k    0     0  1082k      0  0:00:05  0:00:05 --:--:-- 1436k


In [6]:
git status

On branch master

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	NB-core_v4.map.gz

nothing added to commit but untracked files present (use "git add" to track)


Git сообщает, что файл `NB-core_v4.map.gz` ему незнаком (untracked).

---

# Сохранение изменений

Появление в директории нового файла является изменением. Но если `NB-core_v4.map.gz` удалить на данном этапе, то git никак не поможет его восстановить, потому что git не следит за этим файлом. Чтобы добавить этот файл в репозиторий (поручить за ним следить), используется следующая команда:

In [7]:
git add NB-core_v4.map.gz

In [9]:
git status

On branch master

Initial commit

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)

	new file:   NB-core_v4.map.gz



Сохранение изменений в git происходит в два шага: добавление изменения в индекс (index or staging area) и сохранение индекса в виде коммита. Коммит навсегда запоминает изменения, сложенные в индекс.

In [10]:
git commit -m 'Add an example input'

[master (root-commit) 0663f3d] Add an example input
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 NB-core_v4.map.gz


In [11]:
git status

On branch master
nothing to commit, working tree clean


Каждый коммит сопровождается кратким и развёрнутым описаниями изменений, которые он содержит. Краткое описание принято писать в императиве аналогично описаниям автоматических коммитов, которые иногда создаёт сам git.

---

## Инициализация скрипта

In [12]:
touch count.sh

### count.sh

In [13]:
git status

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	count.sh

nothing added to commit but untracked files present (use "git add" to track)


In [14]:
git add count.sh

In [15]:
git commit -m 'Init the script'

[master 12ac34a] Init the script
 1 file changed, 1 insertion(+)
 create mode 100644 count.sh


---

### count.sh

In [ ]:
#!/usr/bin/env bash

awk < $1 '{ count[$1]++; } END { for (i in count) { print i, count[i] }}'

---

# Подготовка файлов

In [16]:
gzip -dc 'NB-core_v4.map.gz' > rice.map

In [17]:
./count.sh

bash: ./count.sh: Permission denied


: 126

In [18]:
chmod +x ./count.sh

---

In [19]:
./count.sh rice.map

2 84248
3 74582
4 98579
5 82672
6 81926
7 79933
8 90302
9 62668
10 73685
11 100070
12 76540
1 90804


---

# Выбор изменений для коммита

In [20]:
git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   count.sh

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	rice.map

no changes added to commit (use "git add" and/or "git commit -a")


В директории произошло два не связанных изменения: извлечение файла из архива и написание скрипта. Сохранять эти два изменения в одном коммите нежелательно, так как для извлечении максимальной пользы из использования git, коммиты следует делать максимально гранулярными.

Промежуточный шаг добавления в индекс существует, чтобы разделить эти два уже случившихся изменения на два коммита.

In [20]:
git add rice.map

In [21]:
git commit -m 'Unzip the MAP file for rice'

[master b25dc2c] Unzip the MAP file for rice
 1 file changed, 996009 insertions(+)
 create mode 100644 rice.map


In [22]:
git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   count.sh

no changes added to commit (use "git add" and/or "git commit -a")


In [23]:
git add count.sh

In [24]:
git commit -m 'Implement counting using awk'

[master 0501706] Implement counting using awk
 1 file changed, 2 insertions(+)
 mode change 100644 => 100755 count.sh


In [25]:
git status

On branch master
nothing to commit, working tree clean


---

# Просмотр истории

In [26]:
git log

commit 050170610a075abf1fc38c7f890e749a2ae523e9
Author: Ivan Dmitrievsky <ivan.dmitrievsky@gmail.com>
Date:   Tue Feb 28 17:33:20 2017 +0300

    Implement counting using awk

commit b25dc2c4483061773a7148766d56ca53e7483daa
Author: Ivan Dmitrievsky <ivan.dmitrievsky@gmail.com>
Date:   Tue Feb 28 17:33:17 2017 +0300

    Unzip the MAP file for rice

commit 12ac34aa45bbe19b130eb2343475d65373c0dffb
Author: Ivan Dmitrievsky <ivan.dmitrievsky@gmail.com>
Date:   Tue Feb 28 17:25:22 2017 +0300

    Init the script

commit 0663f3d5edbf8c098ab8d210efb873dc88aa3d85
Author: Ivan Dmitrievsky <ivan.dmitrievsky@gmail.com>
Date:   Tue Feb 28 17:24:27 2017 +0300

    Add an example input


Команда `log` отображает историю коммитов в репозитории. Каждый коммит имеет уникальный ключ, который используется, чтобы ссылаться на него, дату создания, автора и прочую информацию.

`log` предоставляет большое количество опций для настроек вывода истории.

In [27]:
git log --pretty=oneline

050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice
12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


In [31]:
git log --pretty=oneline --since='5 minutes ago'

050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice


In [32]:
git log --pretty=oneline --graph 

* 050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
* b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice
* 12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
* 0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


---

# Передвижение по истории

Git хранит историю как цепочку коммитов. Так как существует возможность перемещения между коммитами, git имеет специальный указатель на текущий коммит, который называется `HEAD`.

In [36]:
git checkout 12ac3

Previous HEAD position was 0501706... Implement counting using awk
HEAD is now at 12ac34a... Init the script


In [38]:
git status

HEAD detached at 12ac34a
nothing to commit, working tree clean


In [37]:
ls -lah

total 12096
drwxr-xr-x   5 idmit  staff   170B Feb 28 17:38 .
drwxr-xr-x   3 idmit  staff   102B Feb 28 17:05 ..
drwxr-xr-x  12 idmit  staff   408B Feb 28 17:38 .git
-rw-r--r--   1 idmit  staff   5.9M Feb 28 17:11 NB-core_v4.map.gz
-rw-r--r--   1 idmit  staff    20B Feb 28 17:38 count.sh


Так как коммит `12ac3` предшествует извлечению файла с входными данными из архива, файл `rice.map` отсутствует в директории. 

Чтобы вернуться назад, нужно вспомнить ключ последнего коммита в истории, который сейчас в истории не отображается.

In [39]:
git log --pretty=oneline --graph 

* 12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
* 0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


К счастью, git по-умолчанию создаёт и хранит указатель на последний коммит в репозитории. Этот указатель называется `master`.

In [40]:
git checkout master

Previous HEAD position was 12ac34a... Init the script
Switched to branch 'master'


---

# Отмена изменений до индексации

Иногда в репозитории могут происходить нежелательные изменения.

In [42]:
echo 'O-ops!' > count.sh

In [44]:
cat count.sh

O-ops!


In [45]:
git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   count.sh

no changes added to commit (use "git add" and/or "git commit -a")


Git знает, что файл изменился, но ещё не начал сохранять эти изменения — они даже не добавлены в индекс. Чтобы отменить изменение в определённом файле можно снова применить команду `checkout`, которая переключит содержимое файла на сохранённое в репозитории.

In [47]:
git checkout -- count.sh

In [48]:
cat count.sh

#!/usr/bin/env bash

awk < $1 '{ count[$1]++; } END { for (i in count) { print i, count[i] }}'


---

# Отмена изменений после индексации

In [49]:
echo 'O-ops!' > count.sh

In [50]:
cat count.sh

O-ops!


Иногда изменения могут быть добавлены в индекс невовремя, и возникает желание их оттуда извлечь.

In [53]:
git add count.sh

In [54]:
git status

On branch master
Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   count.sh



Тогда команда `checkout` не поможет, потому что git воспринимает версию файла в индексе как самую свежую сохранённую версию файла.

In [55]:
git checkout count.sh

In [56]:
cat count.sh

O-ops!


In [57]:
git status

On branch master
Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   count.sh



Команда `reset` извлекает файл из индекса.

In [58]:
git reset count.sh

Unstaged changes after reset:
M	count.sh


При этом изменения, сделанные в файле, не отменяются.

In [60]:
git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   count.sh

no changes added to commit (use "git add" and/or "git commit -a")


In [61]:
cat count.sh

O-ops!


Непроиндексированные изменения по-прежнему можно отменить командой `checkout`.

In [63]:
git checkout -- count.sh

In [64]:
git status

On branch master
nothing to commit, working tree clean


In [65]:
cat count.sh

#!/usr/bin/env bash

awk < $1 '{ count[$1]++; } END { for (i in count) { print i, count[i] }}'


---

# Безопасная отмена целых коммитов

In [66]:
echo 'O-ops!' > count.sh

In [67]:
cat count.sh

O-ops!


In [68]:
git add count.sh

Случается, что нежелательные изменения оказываются сохранены в коммите.

In [69]:
git commit -m 'Remove the script'

[master 8560d3b] Remove the script
 1 file changed, 1 insertion(+), 3 deletions(-)


Для таких случаев существует команда `revert`, которая создаёт новый коммит, возвращающий состояние директории к состоянию, сохранённому в предшествующем коммите. Чтобы обратить последний коммит, достаточно обратить `HEAD`.

In [70]:
git revert HEAD --no-edit

[master 75c0bea] Revert "Remove the script"
 1 file changed, 3 insertions(+), 1 deletion(-)


In [71]:
cat count.sh

#!/usr/bin/env bash

awk < $1 '{ count[$1]++; } END { for (i in count) { print i, count[i] }}'


In [72]:
git log --pretty=oneline --graph 

* 75c0bea5b33e4cffb7181184a3dfebb51eb35cb4 Revert "Remove the script"
* 8560d3be877ed9749e88c3e816a051da49efc549 Remove the script
* 050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
* b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice
* 12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
* 0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


---

# Внесение изменений в последний коммит

### count.sh

In [ ]:
#!/usr/bin/env bash

# Ivan Dmitrievsky

awk < $1 '{ count[$1]++; } END { for (i in count) { print i, count[i] }}'

In [73]:
git add count.sh

In [74]:
git commit -m 'Add a signature to the script'

[master e7da2e7] Add a signature to the script
 1 file changed, 2 insertions(+)


In [75]:
git log --pretty=oneline --graph 

* e7da2e7bc433857b369eb669bae7fdd698a88228 Add a signature to the script
* 75c0bea5b33e4cffb7181184a3dfebb51eb35cb4 Revert "Remove the script"
* 8560d3be877ed9749e88c3e816a051da49efc549 Remove the script
* 050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
* b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice
* 12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
* 0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


### count.sh

In [ ]:
#!/usr/bin/env bash

# Ivan Dmitrievsky, ivan.dmitrievsky@gmail.com

awk < $1 '{ count[$1]++; } END { for (i in count) { print i, count[i] }}'

In [76]:
git add count.sh

Если изменения в индексе тесно связаны с изменениями в предыдущем коммите, то ключом `--amend` их можно сохранить в предыдущий коммит.

In [77]:
git commit --amend -m 'Add contact info to the script'

[master a0003be] Add contact info to the script
 Date: Tue Feb 28 18:04:59 2017 +0300
 1 file changed, 2 insertions(+)


In [78]:
git log --pretty=oneline --graph 

* a0003be2668162aa1a77ab90118c55017b280762 Add contact info to the script
* 75c0bea5b33e4cffb7181184a3dfebb51eb35cb4 Revert "Remove the script"
* 8560d3be877ed9749e88c3e816a051da49efc549 Remove the script
* 050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
* b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice
* 12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
* 0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


In [79]:
cat count.sh

#!/usr/bin/env bash

# Ivan Dmitrievsky, ivan.dmitrievsky@gmail.com

awk < $1 '{ count[$1]++; } END { for (i in count) { print i, count[i] }}'


---

# Создание ветки

Часто возникает желание работать одновременно над двумя вещами в одном репозитории. При этом естественным образом возрастает опасность внести нежелательные изменения. Если в таком случае продолжать вести историю линейно, то коммиты, соответствующие работе над разными вещами, будут переплетены, что означает, что почти все коммиты будут хранить версию директории в процессе изменения.

Git позволяет избежать этих проблем с помощью веток.

In [80]:
git checkout -b ordered-list

Switched to a new branch 'ordered-list'


In [81]:
git status

On branch ordered-list
nothing to commit, working tree clean


### count.sh | ordered-list

In [ ]:
#!/usr/bin/env bash

# Ivan Dmitrievsky, ivan.dmitrievsky@gmail.com

cut -f 1 $1 | sort | uniq -c | sort -nr

In [82]:
./count.sh rice.map

100070 11
98579 4
90804 1
90302 8
84248 2
82672 5
81926 6
79933 7
76540 12
74582 3
73685 10
62668 9


In [83]:
git add count.sh

In [84]:
git commit -m 'Change the script to print ordered list'

[ordered-list 21fe810] Change the script to print ordered list
 1 file changed, 1 insertion(+), 1 deletion(-)


In [85]:
git status

On branch ordered-list
nothing to commit, working tree clean


In [86]:
git log --pretty=oneline --graph 

* 21fe8109e7c70b856a522868e17419837d258941 Change the script to print ordered list
* a0003be2668162aa1a77ab90118c55017b280762 Add contact info to the script
* 75c0bea5b33e4cffb7181184a3dfebb51eb35cb4 Revert "Remove the script"
* 8560d3be877ed9749e88c3e816a051da49efc549 Remove the script
* 050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
* b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice
* 12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
* 0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


---

# Навигация по веткам

In [87]:
git branch

  master
* ordered-list


In [88]:
git checkout master

Switched to branch 'master'


In [89]:
cat count.sh

#!/usr/bin/env bash

# Ivan Dmitrievsky, ivan.dmitrievsky@gmail.com

awk < $1 '{ count[$1]++; } END { for (i in count) { print i, count[i] }}'


In [90]:
git log --pretty=oneline --graph 

* a0003be2668162aa1a77ab90118c55017b280762 Add contact info to the script
* 75c0bea5b33e4cffb7181184a3dfebb51eb35cb4 Revert "Remove the script"
* 8560d3be877ed9749e88c3e816a051da49efc549 Remove the script
* 050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
* b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice
* 12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
* 0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


In [91]:
git log --pretty=oneline --graph  --all

* 21fe8109e7c70b856a522868e17419837d258941 Change the script to print ordered list
* a0003be2668162aa1a77ab90118c55017b280762 Add contact info to the script
* 75c0bea5b33e4cffb7181184a3dfebb51eb35cb4 Revert "Remove the script"
* 8560d3be877ed9749e88c3e816a051da49efc549 Remove the script
* 050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
* b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice
* 12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
* 0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


Ветка `master` кончается на коммите `a0003`, а ветка `ordered-list` – на `21fe8`.

---

# Слияние веток

После завершения всех изменений, относящихся к ветке `ordered-list`, эти изменения бывает нужно перенести в обратно в ветку `master`.

In [92]:
git status

On branch master
nothing to commit, working tree clean


Ветку `ordered-list` можно «влить» в `master`.

In [94]:
git merge ordered-list

Updating a0003be..21fe810
Fast-forward
 count.sh | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


С тех пор, как ветка `ordered-list` ответвилась от `master`, в `master` не произошло никаких изменений. Поэтому git может применить изменения, сохранённые в коммитах `ordered-list`, напрямую к `master`. Таким образом указатель `master` выровняется с `ordered-list`. Такое слияние называется fast-forward.

In [95]:
cat count.sh

#!/usr/bin/env bash

# Ivan Dmitrievsky, ivan.dmitrievsky@gmail.com

cut -f 1 $1 | sort | uniq -c | sort -nr


In [96]:
git log --pretty=oneline --graph

* 21fe8109e7c70b856a522868e17419837d258941 Change the script to print ordered list
* a0003be2668162aa1a77ab90118c55017b280762 Add contact info to the script
* 75c0bea5b33e4cffb7181184a3dfebb51eb35cb4 Revert "Remove the script"
* 8560d3be877ed9749e88c3e816a051da49efc549 Remove the script
* 050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
* b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice
* 12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
* 0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


---

# Разрешение конфликтов

Гораздо чаще возникают ситуации, в которых ветка `master` успевает обзавестись дополнительными коммитами, которых нет в `ordered-list`.

### count.sh | master

In [ ]:
#!/usr/bin/env bash

# Ivan Dmitrievsky, ivan.dmitrievsky@gmail.com

cut -f 1 $1 | sort | uniq -c | sort --numeric-sort --reverse

In [97]:
git add count.sh

In [98]:
git commit -m 'Replace short options with long ones'

[master 8b7d17f] Replace short options with long ones
 1 file changed, 1 insertion(+), 1 deletion(-)


In [99]:
git checkout ordered-list

Switched to branch 'ordered-list'


### count.sh | ordered-list

In [ ]:
#!/usr/bin/env bash

# Ivan Dmitrievsky, ivan.dmitrievsky@gmail.com

cut -f1 $1 | sort | uniq -c | sort -nr

In [100]:
git add count.sh

In [101]:
git commit -m 'Make cut options more compact'

[ordered-list 3cef93f] Make cut options more compact
 1 file changed, 1 insertion(+), 1 deletion(-)


Команда `log` с ключами `--graph` и `--all` покажет, что истории веток разошлись.

In [105]:
git log --pretty=oneline --graph --all

* 3cef93f64989bdb82af77dbf5d55e4a40df44e0d Make cut options more compact
| * 8b7d17f1e192df3f9c55309fb9e1f02d96c847f4 Replace short options with long ones
|/  
* 21fe8109e7c70b856a522868e17419837d258941 Change the script to print ordered list
* a0003be2668162aa1a77ab90118c55017b280762 Add contact info to the script
* 75c0bea5b33e4cffb7181184a3dfebb51eb35cb4 Revert "Remove the script"
* 8560d3be877ed9749e88c3e816a051da49efc549 Remove the script
* 050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
* b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice
* 12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
* 0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


In [106]:
git checkout master

Switched to branch 'master'


In [107]:
git merge ordered-list 

Auto-merging count.sh
CONFLICT (content): Merge conflict in count.sh
Automatic merge failed; fix conflicts and then commit the result.


: 1

В таком случае при попытке слияния возникает конфликт.

In [108]:
cat count.sh

#!/usr/bin/env bash

# Ivan Dmitrievsky, ivan.dmitrievsky@gmail.com

<<<<<<< HEAD
cut -f 1 $1 | sort | uniq -c | sort --numeric-sort --reverse
cut -f1 $1 | sort | uniq -c | sort -nr
>>>>>>> ordered-list


### count.sh | master

In [ ]:
#!/usr/bin/env bash

# Ivan Dmitrievsky, ivan.dmitrievsky@gmail.com

cut -f1 $1 | sort | uniq -c | sort --numeric-sort --reverse

In [109]:
git add count.sh

In [110]:
git commit -m 'Merge ordered-list into master'

[master 3fcf4f0] Merge ordered-list into master


In [111]:
git log --pretty=oneline --graph

*   3fcf4f0720a58426c068bb32334ff7286aed0a2d Merge ordered-list into master
|\  
| * 3cef93f64989bdb82af77dbf5d55e4a40df44e0d Make cut options more compact
* | 8b7d17f1e192df3f9c55309fb9e1f02d96c847f4 Replace short options with long ones
|/  
* 21fe8109e7c70b856a522868e17419837d258941 Change the script to print ordered list
* a0003be2668162aa1a77ab90118c55017b280762 Add contact info to the script
* 75c0bea5b33e4cffb7181184a3dfebb51eb35cb4 Revert "Remove the script"
* 8560d3be877ed9749e88c3e816a051da49efc549 Remove the script
* 050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
* b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice
* 12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
* 0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


В случаях конфликта существуют альтернативы слиянию, например, команда `rebase`, которые здесь не рассматриваются.

---

# Клонирование репозиториев

Часто возникает потребность в работе над одним и тем же проектом на нескольких компьютерах. Git обладает мощными инструментами для решения задач такого рода.

In [114]:
cd ..

In [115]:
pwd

/Users/idmit/Documents/dev/sandbox/git_workshop/gitstarter


Команда `clone` создаёт клон репозитория. Клон хранит все те же изменения, но не является точной копией. В качестве аргумента команде `clone` чаще всего передаётся URL.

In [116]:
git clone count cloned_repo

Cloning into 'cloned_repo'...
done.


In [117]:
ls -lah

total 0
drwxr-xr-x  4 idmit  staff   136B Feb 28 18:34 .
drwxr-xr-x  5 idmit  staff   170B Feb 28 17:05 ..
drwxr-xr-x  6 idmit  staff   204B Feb 28 18:34 cloned_repo
drwxr-xr-x  6 idmit  staff   204B Feb 28 18:27 count


In [118]:
cd cloned_repo

In [119]:
ls -lah

total 56144
drwxr-xr-x   6 idmit  staff   204B Feb 28 18:34 .
drwxr-xr-x   4 idmit  staff   136B Feb 28 18:34 ..
drwxr-xr-x  12 idmit  staff   408B Feb 28 18:34 .git
-rw-r--r--   1 idmit  staff   5.9M Feb 28 18:34 NB-core_v4.map.gz
-rwxr-xr-x   1 idmit  staff   129B Feb 28 18:34 count.sh
-rw-r--r--   1 idmit  staff    22M Feb 28 18:34 rice.map


In [120]:
git log --pretty=oneline --graph

*   3fcf4f0720a58426c068bb32334ff7286aed0a2d Merge ordered-list into master
|\  
| * 3cef93f64989bdb82af77dbf5d55e4a40df44e0d Make cut options more compact
* | 8b7d17f1e192df3f9c55309fb9e1f02d96c847f4 Replace short options with long ones
|/  
* 21fe8109e7c70b856a522868e17419837d258941 Change the script to print ordered list
* a0003be2668162aa1a77ab90118c55017b280762 Add contact info to the script
* 75c0bea5b33e4cffb7181184a3dfebb51eb35cb4 Revert "Remove the script"
* 8560d3be877ed9749e88c3e816a051da49efc549 Remove the script
* 050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
* b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice
* 12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
* 0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


Содержимое клона совпадает с исходным репозиторием.

---

# Про origin и remote

Любой репозиторий может хранить список удалённых (remote) репозиториев, состояние которых он может отслеживать по запросу пользователя.

Клонированный репозиторий по-умолчанию имеет удалённый репозиторий, который называется `origin`. Это указатель на исходный репозиторий. Git знает, где он находится, потому что его местоположение была передано команде `clone` при создании клона. 

In [121]:
git remote

origin


Информацию об удалённом репозитории можно отобразить с помощью `remote show`.

In [122]:
git remote show origin

* remote origin
  Fetch URL: /Users/idmit/Documents/dev/sandbox/git_workshop/gitstarter/count
  Push  URL: /Users/idmit/Documents/dev/sandbox/git_workshop/gitstarter/count
  HEAD branch: master
  Remote branches:
    master       tracked
    ordered-list tracked
  Local branch configured for 'git pull':
    master merges with remote master
  Local ref configured for 'git push':
    master pushes to master (up to date)


In [123]:
git branch

* master


Если отобразить все ветки, то помимо локальных веток, также отобразятся и ветки удалённых репозиториев.

In [126]:
git branch -a

* master
  remotes/origin/HEAD -> origin/master
  remotes/origin/master
  remotes/origin/ordered-list


---

# Внесение изменений

In [127]:
cd ../count

### count.sh

In [ ]:
#!/usr/bin/env bash

# Ivan Dmitrievsky, ivan.dmitrievsky@gmail.com.
# All rights reserved, 2017.

cut -f1 $1 | sort | uniq -c | sort --numeric-sort --reverse

In [128]:
git add count.sh

In [129]:
git commit -m "Add copyright"

[master 5bf7279] Add copyright
 1 file changed, 2 insertions(+), 1 deletion(-)


In [130]:
git log --pretty=oneline --graph

* 5bf7279f2f8d4281087c4676a8f1eac4cc26b6a5 Add copyright
*   3fcf4f0720a58426c068bb32334ff7286aed0a2d Merge ordered-list into master
|\  
| * 3cef93f64989bdb82af77dbf5d55e4a40df44e0d Make cut options more compact
* | 8b7d17f1e192df3f9c55309fb9e1f02d96c847f4 Replace short options with long ones
|/  
* 21fe8109e7c70b856a522868e17419837d258941 Change the script to print ordered list
* a0003be2668162aa1a77ab90118c55017b280762 Add contact info to the script
* 75c0bea5b33e4cffb7181184a3dfebb51eb35cb4 Revert "Remove the script"
* 8560d3be877ed9749e88c3e816a051da49efc549 Remove the script
* 050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
* b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice
* 12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
* 0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


---

# Принятие изменений

In [131]:
cd ../cloned_repo

In [133]:
git log --pretty=oneline --graph --all

*   3fcf4f0720a58426c068bb32334ff7286aed0a2d Merge ordered-list into master
|\  
| * 3cef93f64989bdb82af77dbf5d55e4a40df44e0d Make cut options more compact
* | 8b7d17f1e192df3f9c55309fb9e1f02d96c847f4 Replace short options with long ones
|/  
* 21fe8109e7c70b856a522868e17419837d258941 Change the script to print ordered list
* a0003be2668162aa1a77ab90118c55017b280762 Add contact info to the script
* 75c0bea5b33e4cffb7181184a3dfebb51eb35cb4 Revert "Remove the script"
* 8560d3be877ed9749e88c3e816a051da49efc549 Remove the script
* 050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
* b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice
* 12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
* 0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


In [134]:
cat count.sh

#!/usr/bin/env bash

# Ivan Dmitrievsky, ivan.dmitrievsky@gmail.com

cut -f1 $1 | sort | uniq -c | sort --numeric-sort --reverse


Изменения не переносятся в клонированный репозиторий автоматически. Чтобы скачать изменения, произошедшие в удалённых репозиториях, используется команда `fetch`.

In [135]:
git fetch

remote: Counting objects: 3, done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0)
Unpacking objects: 100% (3/3), done.
From /Users/idmit/Documents/dev/sandbox/git_workshop/gitstarter/count
   3fcf4f0..5bf7279  master     -> origin/master


In [136]:
git log --pretty=oneline --graph  --all

* 5bf7279f2f8d4281087c4676a8f1eac4cc26b6a5 Add copyright
*   3fcf4f0720a58426c068bb32334ff7286aed0a2d Merge ordered-list into master
|\  
| * 3cef93f64989bdb82af77dbf5d55e4a40df44e0d Make cut options more compact
* | 8b7d17f1e192df3f9c55309fb9e1f02d96c847f4 Replace short options with long ones
|/  
* 21fe8109e7c70b856a522868e17419837d258941 Change the script to print ordered list
* a0003be2668162aa1a77ab90118c55017b280762 Add contact info to the script
* 75c0bea5b33e4cffb7181184a3dfebb51eb35cb4 Revert "Remove the script"
* 8560d3be877ed9749e88c3e816a051da49efc549 Remove the script
* 050170610a075abf1fc38c7f890e749a2ae523e9 Implement counting using awk
* b25dc2c4483061773a7148766d56ca53e7483daa Unzip the MAP file for rice
* 12ac34aa45bbe19b130eb2343475d65373c0dffb Init the script
* 0663f3d5edbf8c098ab8d210efb873dc88aa3d85 Add an example input


Работа с удалёнными ветками аналогична работе с локальными ветками (удалённые ветки это лишь название особых локальных веток). Чтобы добавить изменения в главную ветку, нужно выполнить слияние.

In [137]:
git merge origin/master

Updating 3fcf4f0..5bf7279
Fast-forward
 count.sh | 3 ++-
 1 file changed, 2 insertions(+), 1 deletion(-)


In [138]:
cat count.sh

#!/usr/bin/env bash

# Ivan Dmitrievsky, ivan.dmitrievsky@gmail.com.
# All rights reserved, 2017.

cut -f1 $1 | sort | uniq -c | sort --numeric-sort --reverse


Эти команды так часто выполнятются одновременно, что существует команда `pull`, которая выполняет и `fetch`, и слияние.

---

# Отправка изменений

Репозиторий клон может скачивать изменения, происходящие в исходном репозитории. Как осуществить передачу изменений в обратном направлении?

Можно добавить клона в исходный репозитории в качестве remote, так пара репозиториев может обмениваться изменениями. Но что если мы хотим работать над проектом командой из десяти человек? Как всем десяти обмениваться изменениями между собой?

Обычно эта задача решается с помощью специально выделенного централизированного репозитория, который служит для синхронизации всех остальных. Такой репозиторий помещается на сервере, который всегда доступен, чтобы всегда существовала возможность синхронизации.

In [148]:
cd ..

In [149]:
pwd

/Users/idmit/Documents/dev/sandbox/git_workshop/gitstarter


Однако, поскольку этот репозиторий служит только для синхронизации, и никто не работает в нём с реальными файлами, то ему необходимо хранить лишь изменения, которые он получает и отправляет. Обычно git хранит информацию об изменениях в поддиректории `.git` репозитория, но существует команда, создающая «голый» репозиторий, который может хранить только изменения.

In [150]:
git init --bare shared.git

Initialized empty Git repository in /Users/idmit/Documents/dev/sandbox/git_workshop/gitstarter/shared.git/


После исполнения этой команды рабочий каталог не становится репозиторием. Репозитоием является `shared.git`.

In [151]:
ls -lah shared.git

total 24
drwxr-xr-x   9 idmit  staff   306B Feb 28 19:04 .
drwxr-xr-x   5 idmit  staff   170B Feb 28 19:04 ..
-rw-r--r--   1 idmit  staff    23B Feb 28 19:04 HEAD
-rw-r--r--   1 idmit  staff   111B Feb 28 19:04 config
-rw-r--r--   1 idmit  staff    73B Feb 28 19:04 description
drwxr-xr-x  12 idmit  staff   408B Feb 28 19:04 hooks
drwxr-xr-x   3 idmit  staff   102B Feb 28 19:04 info
drwxr-xr-x   4 idmit  staff   136B Feb 28 19:04 objects
drwxr-xr-x   4 idmit  staff   136B Feb 28 19:04 refs


In [152]:
cd count

In [153]:
git remote add shared ../shared.git

In [154]:
cd ../cloned_repo

In [155]:
git remote add shared ../shared.git

### count.sh | cloned_repo

In [ ]:
#!/usr/bin/env bash

# Ivan Dmitrievsky, ivan.dmitrievsky@gmail.com.
# John Doe, john.doe@email.com
# All rights reserved, 2017.

cut -f1 $1 | sort | uniq -c | sort --numeric-sort --reverse

In [156]:
git add count.sh

In [157]:
git commit -m 'Add John Doe as an author'

[master 382d261] Add John Doe as an author
 1 file changed, 1 insertion(+)


In [158]:
git push shared master

Counting objects: 37, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (34/34), done.
Writing objects: 100% (37/37), 11.75 MiB | 3.40 MiB/s, done.
Total 37 (delta 8), reused 0 (delta 0)
To ../shared.git
 * [new branch]      master -> master


In [159]:
cd ../count

In [160]:
git pull shared master

remote: Counting objects: 3, done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0)
Unpacking objects: 100% (3/3), done.
From ../shared
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> shared/master
Updating 5bf7279..382d261
Fast-forward
 count.sh | 1 +
 1 file changed, 1 insertion(+)


In [161]:
cat count.sh

#!/usr/bin/env bash

# Ivan Dmitrievsky, ivan.dmitrievsky@gmail.com.
# John Doe, john.doe@email.com
# All rights reserved, 2017.

cut -f1 $1 | sort | uniq -c | sort --numeric-sort --reverse


---

# Шаблоны использования Git

Существует множество сервисов, готовых выступать в роли сервера для группы репозиториев. Самым популярным является https://github.com.

Чаще всего сначала создаётся локальный репозиторий, в котором ведётся работа. При возникновении необходимости работать в команде создаётся «голый» репозиторий на GitHub. URL этого репозитория добавляется в качестве remote репозитория, в который периодически выполняется `push`.

В случаях, когда с самого начала планируется вести командную разработку, на GitHub сразу создаётся «голый» репозиторий, который затем клонируется на компьютер. Клон уже содержит исходный репозиторий в качестве удалённого.

Также GitHub используется для публикации приложений с открытым исходным кодом. В таком случае другие пользователи могут клонировать ваш репозиторий, а затем предлагать изменения, исправляющие ошибки или добаляющие новую функциональность.